## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, GRU, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import Callback, EarlyStopping

## Data Loading

In [3]:
train_df = pd.read_csv('../data/processed/train_data_processed.csv', index_col='id')
train_df.head()

,title,genre,description,year,description_stemmed,description_stemmed_string
id,,,,,,
1,Oscar et la dame rose,drama,Listening in to a conversation between his do...,2009,"['listen', 'convers', 'doctor', 'parent', '10-...",listen convers doctor parent 10-year-old oscar...
2,Cupid,thriller,A brother and sister with a past incestuous r...,1997,"['brother', 'sister', 'past', 'incestu', 'rela...",brother sister past incestu relationship curre...
3,"Young, Wild and Wonderful",adult,As the bus empties the students for their fie...,1980,"['bu', 'empti', 'student', 'field', 'trip', 'm...",bu empti student field trip museum natur histo...
4,The Secret Sin,drama,To help their unemployed father make ends mee...,1915,"['help', 'unemploy', 'father', 'make', 'end', ...",help unemploy father make end meet edith twin ...
5,The Unrecovered,drama,The film's title refers not only to the un-re...,2007,"['film', ""'s"", 'titl', 'refer', 'un-recov', 'b...",film 's titl refer un-recov bodi ground zero a...


In [4]:
unique_genres = train_df['genre'].unique()
genre_encoding = dict([(unique_genres[i], i) for i in range(unique_genres.shape[0])])
train_df['genre'] = train_df['genre'].map(genre_encoding)
train_df = train_df.sample(frac=1)
train_df

,title,genre,description,year,description_stemmed,description_stemmed_string
id,,,,,,
49920,A Mile from Home,0,A Mile from Home chronicles the life of a uni...,2013,"['mile', 'home', 'chronicl', 'life', 'univers'...",mile home chronicl life univers student jude o...
25445,"""Super Adventure Theater""",18,The show was aired from 20 September 1959 til...,1959,"['show', 'air', '20', 'septemb', '1959', 'till...",show air 20 septemb 1959 till 1967 host claud ...
46954,The Hungry Ghosts,0,"According to Buddhists, hungry ghosts are the...",2009,"['accord', 'buddhist', 'hungri', 'ghost', 'dea...",accord buddhist hungri ghost dead bid farewel ...
38586,Vom Hirschkäfer zum Hakenkreuz,3,"The film narrates in a richly detailed, assoc...",2002,"['film', 'narrat', 'richli', 'detail', 'associ...",film narrat richli detail associ montag boundl...
20785,White T,4,Herbert & Henry (Jerod and Jamal Mixon) are t...,2013,"['herbert', 'henri', 'jerod', 'jamal', 'mixon'...",herbert henri jerod jamal mixon twin brother d...
...,...,...,...,...,...,...
7690,Faceless (Afghan,10,"Sameer is a middle-class young man, who lives...",stan,"['sameer', 'middle-class', 'young', 'man', 'li...",sameer middle-class young man live younger bro...
10391,"""Aveux""",0,Simon Laplante was once Carl. After running a...,2009,"['simon', 'laplant', 'carl', 'run', 'away', 'f...",simon laplant carl run away famili build new l...
5521,Mother Tongue (201,3,This is a story about a man who risked his li...,/III,"['stori', 'man', 'risk', 'life', 'save', 'cult...",stori man risk life save cultur heritag langua...


In [5]:
num_labels = train_df['genre'].nunique()
num_labels

27

In [6]:
x = train_df['description_stemmed_string'].values
y = pd.get_dummies(train_df['genre']).values

In [7]:
train_set_size = 45000
val_set_size = 3000
test_set_size = 5000

x_train = x[:train_set_size]
x_val = x[train_set_size:train_set_size+val_set_size]
x_test = x[train_set_size+val_set_size:train_set_size+val_set_size+test_set_size]

y_train = y[:train_set_size]
y_val = y[train_set_size:train_set_size+val_set_size]
y_test = y[train_set_size+val_set_size:train_set_size+val_set_size+test_set_size]

## Tokenization

In [8]:
num_words = 2**9
max_len = 2**5

tokenizer = Tokenizer(num_words=num_words, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(x_train)

x_train = tokenizer.texts_to_sequences(x_train)
x_train = pad_sequences(x_train, maxlen=max_len)

x_val = tokenizer.texts_to_sequences(x_val)
x_val = pad_sequences(x_val, maxlen=max_len)

x_test = tokenizer.texts_to_sequences(x_test)
x_test = pad_sequences(x_test, maxlen=max_len)

## NN Training

In [9]:
# model = load_model('../models/nn_model')
model = Sequential()
model.add(Dense(128))
model.add(Dropout(0.1))
model.add(Dense(256))
model.add(Dropout(0.1))
model.add(Dense(256))
model.add(Dropout(0.1))
model.add(Dense(128))
model.add(Dropout(0.1))
model.add(Dense(num_labels, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Metal device set to: Apple M1 Pro


2022-10-24 20:59:55.879348: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-10-24 20:59:55.879448: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [10]:
my_callbacks  = [EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=5,
                              mode='auto')]

history = model.fit(x_train, y_train,
                    epochs=50, batch_size=32,
                    validation_data=(x_val, y_val),
                    callbacks=my_callbacks,
                    verbose=1
                   )

Epoch 1/50


2022-10-24 20:59:56.040223: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


   1/1407 [..............................] - ETA: 13:30 - loss: 386.1678 - accuracy: 0.0000e+00

2022-10-24 20:59:56.443243: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1407/1407 [==============================] - ETA: 0s - loss: 25.2206 - accuracy: 0.1483

2022-10-24 21:00:11.505856: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1407/1407 [==============================] - 16s 11ms/step - loss: 25.2206 - accuracy: 0.1483 - val_loss: 3.3662 - val_accuracy: 0.2413
Epoch 2/50
1407/1407 [==============================] - 15s 11ms/step - loss: 3.8327 - accuracy: 0.1741 - val_loss: 2.5143 - val_accuracy: 0.2467
Epoch 3/50
1407/1407 [==============================] - 14s 10ms/step - loss: 2.8238 - accuracy: 0.2085 - val_loss: 2.4465 - val_accuracy: 0.2473
Epoch 4/50
1407/1407 [==============================] - 15s 10ms/step - loss: 2.5621 - accuracy: 0.2343 - val_loss: 2.4108 - val_accuracy: 0.2447
Epoch 5/50
1407/1407 [==============================] - 15s 10ms/step - loss: 2.4681 - accuracy: 0.2456 - val_loss: 2.3840 - val_accuracy: 0.2570
Epoch 6/50
1407/1407 [==============================] - 15s 10ms/step - loss: 2.4327 - accuracy: 0.2488 - val_loss: 2.4033 - val_accuracy: 0.2467
Epoch 7/50
1407/1407 [==============================] - 15s 11ms/step - loss: 2.4119 - accuracy: 0.2495 - val_loss: 2.4129 - val_accur

## NN Evaluation

In [11]:
model.evaluate(x_test, y_test)

157/157 [==============================] - 1s 8ms/step - loss: 2.3426 - accuracy: 0.2356


[2.3425538539886475, 0.23560000956058502]

In [12]:
model.save('../models/nn_model')

INFO:tensorflow:Assets written to: ../models/nn_model/assets


## GRU Training

In [13]:
EMBEDDING_DIM = 2**5

# model = load_model('../models/gru_model')
model = Sequential()
model.add(Embedding(num_words, EMBEDDING_DIM, input_length=x_train.shape[1]))
model.add(GRU(256, dropout=0.1))
model.add(Dense(num_labels, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
my_callbacks  = [EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=5,
                              mode='auto')]

history = model.fit(x_train, y_train,
                    epochs=20, batch_size=32,
                    validation_data=(x_val, y_val),
                    callbacks=my_callbacks,
                    verbose=1
                   )

Epoch 1/20


2022-10-24 21:04:39.437947: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-24 21:04:39.583586: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


   1/1407 [..............................] - ETA: 31:31 - loss: 3.2958 - accuracy: 0.0938

2022-10-24 21:04:39.717449: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1407/1407 [==============================] - ETA: 0s - loss: 2.0907 - accuracy: 0.3818

2022-10-24 21:05:09.201940: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-24 21:05:09.255318: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1407/1407 [==============================] - 32s 22ms/step - loss: 2.0907 - accuracy: 0.3818 - val_loss: 1.9372 - val_accuracy: 0.4403
Epoch 2/20
1407/1407 [==============================] - 29s 21ms/step - loss: 1.8477 - accuracy: 0.4629 - val_loss: 1.8356 - val_accuracy: 0.4613
Epoch 3/20
1407/1407 [==============================] - 29s 20ms/step - loss: 1.7713 - accuracy: 0.4774 - val_loss: 1.7645 - val_accuracy: 0.4793
Epoch 4/20
1407/1407 [==============================] - 27s 19ms/step - loss: 1.7106 - accuracy: 0.4910 - val_loss: 1.7252 - val_accuracy: 0.4940
Epoch 5/20
1407/1407 [==============================] - 28s 20ms/step - loss: 1.6713 - accuracy: 0.5010 - val_loss: 1.7064 - val_accuracy: 0.4983
Epoch 6/20
1407/1407 [==============================] - 28s 20ms/step - loss: 1.6433 - accuracy: 0.5057 - val_loss: 1.6816 - val_accuracy: 0.5010
Epoch 7/20
1407/1407 [==============================] - 28s 20ms/step - loss: 1.6314 - accuracy: 0.5106 - val_loss: 1.6782 - val_accura

## GRU Evaluation

In [15]:
model.evaluate(x_test, y_test)

157/157 [==============================] - 2s 11ms/step - loss: 1.7513 - accuracy: 0.4822


[1.751349687576294, 0.482200026512146]

In [16]:
model.save('../models/gru_model')

INFO:tensorflow:Assets written to: ../models/gru_model/assets


INFO:tensorflow:Assets written to: ../models/gru_model/assets


## LSTM Training

In [17]:
EMBEDDING_DIM = 2**5

# model = load_model('../models/lstm_model')
model = Sequential()
model.add(Embedding(num_words, EMBEDDING_DIM, input_length=x_train.shape[1]))
model.add(LSTM(64, dropout=0.1))
model.add(Dense(num_labels, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
my_callbacks  = [EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=3,
                              mode='auto')]

history = model.fit(x_train, y_train,
                    epochs=10, batch_size=32,
                    validation_data=(x_val, y_val),
                    callbacks=my_callbacks,
                    verbose=1
                   )

Epoch 1/10


2022-10-24 21:11:20.415076: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-24 21:11:20.564328: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


   1/1407 [..............................] - ETA: 30:45 - loss: 3.2984 - accuracy: 0.0000e+00

2022-10-24 21:11:20.695268: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1407/1407 [==============================] - ETA: 0s - loss: 2.0954 - accuracy: 0.3953

2022-10-24 21:11:47.327354: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-24 21:11:47.382870: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1407/1407 [==============================] - 29s 20ms/step - loss: 2.0954 - accuracy: 0.3953 - val_loss: 1.9248 - val_accuracy: 0.4427
Epoch 2/10
1407/1407 [==============================] - 28s 20ms/step - loss: 1.8336 - accuracy: 0.4682 - val_loss: 1.8151 - val_accuracy: 0.4693
Epoch 3/10
1407/1407 [==============================] - 27s 19ms/step - loss: 1.7578 - accuracy: 0.4829 - val_loss: 1.7551 - val_accuracy: 0.4817
Epoch 4/10
1407/1407 [==============================] - 27s 19ms/step - loss: 1.6906 - accuracy: 0.4986 - val_loss: 1.7009 - val_accuracy: 0.5047
Epoch 5/10
1407/1407 [==============================] - 27s 19ms/step - loss: 1.6445 - accuracy: 0.5078 - val_loss: 1.6798 - val_accuracy: 0.5017
Epoch 6/10
1407/1407 [==============================] - 28s 20ms/step - loss: 1.6094 - accuracy: 0.5147 - val_loss: 1.6596 - val_accuracy: 0.5070
Epoch 7/10
1407/1407 [==============================] - 30s 21ms/step - loss: 1.5821 - accuracy: 0.5174 - val_loss: 1.6512 - val_accura

## LSTM Evaluation

In [19]:
model.evaluate(x_test, y_test)

157/157 [==============================] - 2s 12ms/step - loss: 1.6350 - accuracy: 0.4990


[1.6349989175796509, 0.49900001287460327]

In [20]:
model.save('../models/lstm_model')

INFO:tensorflow:Assets written to: ../models/lstm_model/assets


INFO:tensorflow:Assets written to: ../models/lstm_model/assets
